# RIPEMD-160 Implementation

A pure Python implementation of the RIPEMD-160 cryptographic hash function with comprehensive testing against the standard library.

## Overview

RIPEMD-160 is a 160-bit cryptographic hash function developed as part of the RIPEMD family. It's widely used in Bitcoin addresses and other cryptocurrency applications. This implementation provides a complete, tested, and compatible version of the algorithm.

## Features

- **Pure Python Implementation**: No external dependencies for core functionality
- **Comprehensive Testing**: Validated against PyCryptodome library with 100+ test cases
- **Flexible Input**: Handles both string and bytes input
- **Standard Compliance**: Fully compliant with RIPEMD-160 specification
- **Performance Tested**: Includes random data testing for reliability

## Functions

### `RIPEMD_160_hash(message_str)`
Main hashing function that returns raw bytes:
- Accepts string or bytes input
- Returns 20-byte hash digest
- Processes message in 512-bit chunks

### `ripemd160_hex(message)`
Convenience function that returns hexadecimal string:
- Accepts string or bytes input
- Returns 40-character lowercase hex string
- Wrapper around `RIPEMD_160_hash()`

### Core Algorithm Functions
- `preprocess(message_bytes)`: Message padding and preprocessing
- `f(j, x, y, z)`: Round function (5 different functions)
- `K(j)`, `k_dash(j)`: Round constants for parallel lines
- `r(j)`, `r_dash(j)`: Message word selection arrays
- `s(j)`, `s_dash(j)`: Rotation amounts for each round
- `rol(x, s)`: 32-bit left rotation

## Algorithm Specifications

RIPEMD-160 parameters:
- **Block size**: 512 bits (64 bytes)
- **Output size**: 160 bits (20 bytes)
- **Rounds**: 80 rounds (2 parallel lines of 80 rounds each)
- **Initial values**: Five 32-bit constants
- **Round functions**: 5 different boolean functions
- **Constants**: 10 round constants (5 per line)

## Testing

The implementation includes extensive testing:

### Test Categories
1. **Predefined Test Cases**: Standard test vectors
2. **Random String Testing**: 50 random strings of varying lengths
3. **Binary Data Testing**: 50 random binary sequences

### Test Results
```
============================================================
!!! ALL TESTS PASSED !!!  custom implementation is correct!
============================================================
```

### Running Tests
The test suite validates against PyCryptodome's RIPEMD160 implementation:
```python
from Crypto.Hash import RIPEMD160
# Tests compare custom implementation with library results
```

## Example Outputs

```python
# Empty string
ripemd160_hex("")
# Output: 9c1185a5c5e9fc54612808977ee8f548b2258d31

# Simple string  
ripemd160_hex("abc")
# Output: 8eb208f7e05d987a9b044a8e98c6b087f15a0bfc

# Longer string
ripemd160_hex("The quick brown fox jumps over the lazy dog")
# Output: 37f332f68db77bd9d7edd4969571ad671cf9dd3b
```

## Security & Applications

RIPEMD-160 is considered cryptographically secure and is used in:
- **Bitcoin**: For generating Bitcoin addresses
- **Cryptocurrency**: Various blockchain applications  
- **Digital Signatures**: As part of signature algorithms
- **Data Integrity**: File and message verification

## Dependencies

### Testing (Optional)
- **PyCryptodome**: For validation testing
- **random**: For generating test cases (built-in)

## Standards Compliance

This implementation follows:
- Original RIPEMD-160 specification
- Standard test vectors
- Consistent with reference implementations
- Compatible with existing RIPEMD-160 usage

In [54]:
def preprocess(message_bytes):
    if(not isinstance(message_bytes, bytes)):
        raise TypeError("Input must be of type bytes")
    m = bytearray(message_bytes)
    bit_length = len(m) * 8

    # Append 0x80 (10000000 in binary)
    m.append(0x80)

    # Compute number of zero bytes to append
    padding_len = (56 - (len(m) % 64)) % 64
    m.extend([0x00] * padding_len)

    # Append original bit length as a 64-bit little-endian integer
    m.extend(bit_length.to_bytes(8, byteorder='little'))

    return m

In [55]:
def f(j,x,y,z):
    if(j>=0 and j<=15):
        return x^ y ^ z
    elif(j>=16 and j<=31):
        return (x & y) | (~x & z)   
    elif(j>=32 and j<=47):
        return (x | ~y) ^ z
    elif(j>=48 and j<=63):
        return (x & z) | (y & ~z)
    elif(j>=64 and j<=79):
        return x ^ (y | ~z)

In [56]:
def K(j):
    if(j>=0 and j<=15):
        return 0x00000000
    elif(j>=16 and j<=31):
        return 0x5A827999
    elif(j>=32 and j<=47):
        return 0x6ED9EBA1
    elif(j>=48 and j<=63):
        return 0x8F1BBCDC
    elif(j>=64 and j<=79):
        return 0xA953FD4E
def k_dash(j):
    if(j>=0 and j<=15):
        return 0x50A28BE6
    elif(j>=16 and j<=31):
        return 0x5C4DD124
    elif(j>=32 and j<=47):
        return 0x6D703EF3
    elif(j>=48 and j<=63):
        return 0x7A6D76E9
    elif(j>=64 and j<=79):
        return 0x00000000   

In [57]:
def r(j):
    if(j>=0 and j<=15):
        return j
    elif(j>=16 and j<=31):
        return [7, 4, 13, 1, 10, 6, 15, 3, 12, 0, 9, 5, 2, 14, 11, 8][j-16]
    elif(j>=32 and j<=47):
        return [3, 10, 14, 4, 9, 15, 8, 1, 2, 7, 0, 6, 13, 11, 5, 12][j-32]
    elif(j>=48 and j<=63):
        return [1, 9, 11, 10, 0, 8, 12, 4, 13, 3, 7, 15, 14, 5, 6, 2][j-48]
    elif(j>=64 and j<=79):
        return [4, 0, 5, 9, 7, 12, 2, 10, 14, 1, 3, 8, 11, 6, 15 ,13][j-64]
def r_dash(j):
    if(j>=0 and j<=15):
        return [5, 14, 7, 0, 9, 2, 11, 4, 13, 6, 15, 8, 1, 10, 3, 12][j]
    elif(j>=16 and j<=31):
        return [6, 11, 3, 7, 0, 13, 5, 10, 14, 15, 8, 12, 4, 9, 1, 2][j-16]
    elif(j>=32 and j<=47):
        return [15, 5, 1, 3, 7, 14, 6, 9, 11, 8, 12, 2, 10, 0, 4, 13][j-32]
    elif(j>=48 and j<=63):
        return [8, 6, 4, 1, 3, 11, 15, 0, 5, 12, 2, 13, 9, 7, 10, 14][j-48]
    elif(j>=64 and j<=79):
        return [12,15 ,10 ,4 ,1 ,5 ,8 ,7 ,6 ,2 ,13 ,14 ,0 ,3 ,9 ,11][j-64]

In [58]:
def s(j):
    if(j>=0 and j<=15):
        return [11, 14, 15, 12, 5, 8, 7, 9, 11, 13, 14, 15, 6, 7, 9, 8][j]
    elif(j>=16 and j<=31):
        return [7, 6, 8, 13, 11, 9, 7, 15, 7, 12, 15, 9, 11, 7, 13, 12][j-16]
    elif(j>=32 and j<=47):
        return [11, 13, 6, 7, 14, 9, 13, 15, 14, 8, 13, 6, 5, 12, 7, 5][j-32]
    elif(j>=48 and j<=63):
        return [11,12 ,14 ,15 ,14 ,15 ,9 ,8 ,9 ,14 ,5 ,6 ,8 ,6 ,5 ,12][j-48]
    elif(j>=64 and j<=79):
        return [9 ,15 ,5 ,11 ,6 ,8 ,13 ,12 ,5 ,12 ,13 ,14 ,11 ,8 ,5 ,6][j-64]
def s_dash(j):
    if(j>=0 and j<=15):
        return [8, 9, 9, 11, 13, 15, 15, 5, 7, 7, 8, 11, 14, 14, 12, 6][j]
    elif(j>=16 and j<=31):
        return [9, 13, 15, 7, 12, 8, 9, 11, 7, 7, 12, 7, 6, 15, 13, 11][j-16]
    elif(j>=32 and j<=47):
        return [9, 7, 15, 11, 8, 6, 6, 14, 12, 13, 5, 14, 13, 13, 7, 5][j-32]
    elif(j>=48 and j<=63):
        return [15 ,5 ,8 ,11 ,14 ,14 ,6 ,14 ,6 ,9 ,12 ,9 ,12 ,5 ,15 ,8][j-48]
    elif(j>=64 and j<=79):
        return [8 ,5 ,12 ,9 ,12 ,5 ,14 ,6 ,8 ,13 ,6 ,5 ,15 ,13 ,11 ,11][j-64]    

In [59]:
h0 = 0x67452301
h1 = 0xEFCDAB89
h2 = 0x98BADCFE
h3 = 0x10325476
h4 = 0xC3D2E1F0

In [60]:
"""
rols denotes cyclic left shift (rotate) over s bit positions.
"""
def rol(x, s):
    return ((x << s) | (x >> (32 - s))) & 0xFFFFFFFF

In [61]:
def RIPEMD_160_hash(message_str):
    # Initial hash values
    h0 = 0x67452301
    h1 = 0xEFCDAB89
    h2 = 0x98BADCFE
    h3 = 0x10325476
    h4 = 0xC3D2E1F0
    if isinstance(message_str, str):
        message = message_str.encode("utf-8")
    else:
        message = message_str
    padded_message = preprocess(message)
    
    # Process the message in successive 512-bit chunks
    for i in range(0, len(padded_message), 64):
        block = padded_message[i:i+64]
        
        # Break chunk into sixteen 32-bit little-endian words
        X = []
        for j in range(16):
            X.append(int.from_bytes(block[j*4:j*4+4], byteorder='little'))
        
        # Initialize hash value for this chunk
        A = h0; B = h1; C = h2; D = h3; E = h4
        A_prime = h0; B_prime = h1; C_prime = h2; D_prime = h3; E_prime = h4
        
        # Main loop
        for j in range(80):
            # Left line
            T = (A + f(j, B, C, D) + X[r(j)] + K(j)) & 0xFFFFFFFF
            T = rol(T, s(j))
            T = (T + E) & 0xFFFFFFFF
            A = E
            E = D
            D = rol(C, 10)
            C = B
            B = T
            
            # Right line
            T_prime = (A_prime + f(79-j, B_prime, C_prime, D_prime) + X[r_dash(j)] + k_dash(j)) & 0xFFFFFFFF
            T_prime = rol(T_prime, s_dash(j))
            T_prime = (T_prime + E_prime) & 0xFFFFFFFF
            A_prime = E_prime
            E_prime = D_prime
            D_prime = rol(C_prime, 10)
            C_prime = B_prime
            B_prime = T_prime
        
        # Combine results
        T = (h1 + C + D_prime) & 0xFFFFFFFF
        h1 = (h2 + D + E_prime) & 0xFFFFFFFF
        h2 = (h3 + E + A_prime) & 0xFFFFFFFF
        h3 = (h4 + A + B_prime) & 0xFFFFFFFF
        h4 = (h0 + B + C_prime) & 0xFFFFFFFF
        h0 = T
    
    # Produce the final hash value as a 160-bit number (20 bytes)
    return (h0.to_bytes(4, byteorder='little') + 
            h1.to_bytes(4, byteorder='little') + 
            h2.to_bytes(4, byteorder='little') + 
            h3.to_bytes(4, byteorder='little') + 
            h4.to_bytes(4, byteorder='little'))

def ripemd160_hex(message):
    """Return RIPEMD-160 hash as hexadecimal string"""
    return RIPEMD_160_hash(message).hex()

In [63]:
import random
import string
from Crypto.Hash import RIPEMD160
import random
import string
from Crypto.Hash import RIPEMD160
def generate_random_string(length):
    """Generate a random string of given length"""
    return ''.join(random.choices(string.ascii_letters + string.digits + string.punctuation + ' ', k=length))
def test_custom_vs_library():
    """Test custom implementation against library implementation"""
    print("RIPEMD-160 Implementation Comparison Test")
    print("=" * 60)
    
    # Test with predefined cases first
    test_cases = [
        "",
        "a",
        "abc", 
        "Hello",
        "Hello, World!",
        "The quick brown fox jumps over the lazy dog",
        "1234567890" * 10,  # 100 characters
    ]
    
    print("Testing with predefined cases:")
    print("-" * 40)
    
    all_passed = True
    for i, test_case in enumerate(test_cases):
        # Custom implementation
        custom_result = ripemd160_hex(test_case)
        
        # Library implementation
        h = RIPEMD160.new()
        h.update(test_case.encode('utf-8'))
        library_result = h.hexdigest()
        
        # Compare results
        match = custom_result == library_result
        status = "✓ PASS" if match else "✗ FAIL"
        
        print(f"Test {i+1:2d}: '{test_case[:30]}{'...' if len(test_case) > 30 else ''}' -> {status}")
        print(f"         Custom:  {custom_result}")
        print(f"         Library: {library_result}")
        print()
        
        if not match:
            all_passed = False
    
    # Test with random strings
    print("Testing with random strings:")
    print("-" * 40)
    
    for i in range(50):
        # Generate random string of random length (1 to 200 characters)
        length = random.randint(1, 200)
        test_string = generate_random_string(length)
        
        # Custom implementation
        custom_result = ripemd160_hex(test_string)
        
        # Library implementation
        h = RIPEMD160.new()
        h.update(test_string.encode('utf-8'))
        library_result = h.hexdigest()
        
        # Compare results
        match = custom_result == library_result
        status = "✓ PASS" if match else "✗ FAIL"
        
        print(f"Random {i+1:2d} (len={length:3d}): {status}")
        if not match:
            print(f"         String:  '{test_string[:50]}{'...' if len(test_string) > 50 else ''}' ")
            print(f"         Custom:  {custom_result}")
            print(f"         Library: {library_result}")
            all_passed = False
        
    print()
    def generate_random_binary_data(length):
     """Generate random binary data"""
     return bytes([random.randint(0, 255) for _ in range(length)])
    print("Testing with random binary data:")   
    print("-" * 40)
    for i in range(50):
        # Generate random binary data of random length (1 to 200 bytes)
        length = random.randint(1, 200)
        test_data = generate_random_binary_data(length)
        
        # Custom implementation
        custom_result = ripemd160_hex(test_data)
        
        # Library implementation
        h = RIPEMD160.new()
        h.update(test_data)
        library_result = h.hexdigest()
        
        # Compare results
        match = custom_result == library_result
        status = "✓ PASS" if match else "✗ FAIL"
        
        print(f"Random Binary {i+1:2d} (len={length:3d}): {status}")
        if not match:
            print(f"         Data:    {test_data[:50]}{'...' if len(test_data) > 50 else ''}")
            print(f"         Custom:  {custom_result}")
            print(f"         Library: {library_result}")
            all_passed = False
    print()
    print("=" * 60)
    if all_passed:
        print("!!! ALL TESTS PASSED !!!  custom implementation is correct!")
    else:
        print(" Some tests failed . re-check")
    print("=" * 60)

if __name__ == "__main__":
    random.seed(42)    
    test_custom_vs_library()

RIPEMD-160 Implementation Comparison Test
Testing with predefined cases:
----------------------------------------
Test  1: '' -> ✓ PASS
         Custom:  9c1185a5c5e9fc54612808977ee8f548b2258d31
         Library: 9c1185a5c5e9fc54612808977ee8f548b2258d31

Test  2: 'a' -> ✓ PASS
         Custom:  0bdc9d2d256b3ee9daae347be6f4dc835a467ffe
         Library: 0bdc9d2d256b3ee9daae347be6f4dc835a467ffe

Test  3: 'abc' -> ✓ PASS
         Custom:  8eb208f7e05d987a9b044a8e98c6b087f15a0bfc
         Library: 8eb208f7e05d987a9b044a8e98c6b087f15a0bfc

Test  4: 'Hello' -> ✓ PASS
         Custom:  d44426aca8ae0a69cdbc4021c64fa5ad68ca32fe
         Library: d44426aca8ae0a69cdbc4021c64fa5ad68ca32fe

Test  5: 'Hello, World!' -> ✓ PASS
         Custom:  527a6a4b9a6da75607546842e0e00105350b1aaf
         Library: 527a6a4b9a6da75607546842e0e00105350b1aaf

Test  6: 'The quick brown fox jumps over...' -> ✓ PASS
         Custom:  37f332f68db77bd9d7edd4969571ad671cf9dd3b
         Library: 37f332f68db77bd9d7edd496957